In [2]:
from functools import partial
from collections import namedtuple
import os
from pytorch_toolbelt.utils import read_rgb_image
from predictor import FaceMeshPredictor
import cv2
import os
import mediapipe as mp
from model_training.utils import load_indices_from_npy
from utils import get_relative_path
import numpy as np

/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
objc[6269]: Class CaptureDelegate is implemented in both /Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16450e538) and /Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x28cb48860). One of the two will be used. Which one is undefined.
objc[6269]: Class CVWindow is implemented in both /Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16450e588) and /Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x28bfd8a68). One of the two will be used. Which one is undefined.
objc[6269]: Class CVView is implemented in bot

In [3]:
## this functions take an image as input and returns it's 68 landmark coordinates using dadnet network. 
# For more number of landmarks see run_dad_net_more_points function below
def run_dad_net(image_path):
    image = read_rgb_image(image_path)
    predictor = FaceMeshPredictor.dad_3dnet()
    predictions = predictor(image)
    print(type(predictions))
    coordinates = predictions['points']
    x_pred,y_pred = coordinates[:,0], coordinates[:,1]
    return x_pred, y_pred

In [13]:
# this function uses mediapipe api to detect face and saves the images after cropping only the bounding box
def mediapipe_face_detection(data_path, output_path):
    mp_face_detection = mp.solutions.face_detection
    mp_drawing = mp.solutions.drawing_utils

    #IMAGE_FILES = ["../Dataset/H3DS_sample/sample2/original/0/img_0009.jpg"]
    IMAGE_FILES = []

    files = os.listdir(data_path)

    for file in files:
        curr_image = os.path.join(data_path, file)
        if (file.split('.')[-1] == 'jpg'):
            IMAGE_FILES.append(curr_image)

    #print(len(IMAGE_FILES))
    bboxes = {}

    with mp_face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5) as face_detection:
        for idx, file in enumerate(IMAGE_FILES):
            #print("sancjnajcdd")
            image = cv2.imread(file)
            # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
            results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            if not results.detections:
                continue
            annotated_image = image.copy()

            for detection in results.detections:
                """print('Nose tip:')
                print(mp_face_detection.get_key_point(
                    detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))"""
                mp_drawing.draw_detection(annotated_image, detection)
            curr_bbox = results.detections[0].location_data.relative_bounding_box

            if (curr_bbox.xmin < 0 or curr_bbox.ymin < 0):
                continue
            results.detections[0].location_data.relative_bounding_box
            output_image = os.path.join(output_path, os.path.basename(file))
            filename = os.path.basename(file)
            # get the bounding box parameters which is returned by media pipe api
            bboxes[filename] = results.detections[0].location_data.relative_bounding_box
            #print(output_image)
            #print(bboxes[filename])
            xmin = int(bboxes[filename].xmin*image.shape[1]) ## finds the xmin of bounding box
            ymin = int(bboxes[filename].ymin*image.shape[0]) ## fidn the ymin of bounding box
            w,h = int(bboxes[filename].width*image.shape[1]), int(bboxes[filename].height*image.shape[0]) # find the width and height of bounding box
            detected_portion = image[ymin:ymin+h, xmin:xmin+w] # find the detected region in the image
            #print(detected_portion)
            #print(output_image)
            cv2.imwrite(output_image, detected_portion) #save the detected portion
        
        return bboxes

In [5]:
# this function takes as original image with a face in it and corresponding detected face images (generated using above mediapipe function).
# it runs the DadNet on detected face and find the coordiantes on original image.

def run_dadnet_on_large_images(original_images, data_path, output_path, bboxes):
    #data_path is the path to the detected faces images (not the full image)
    IMAGE_FILES = []

    files = os.listdir(data_path)

    for file in files:
        curr_image = os.path.join(data_path, file)
        IMAGE_FILES.append(curr_image)

    for idx, file in enumerate(IMAGE_FILES):
        pred_x,pred_y = run_dad_net(file)

        filename = os.path.basename(file)
        original_image = os.path.join(original_images, filename)
        image = cv2.imread(original_image)
        xmin = int(bboxes[filename].xmin*image.shape[1]) # find the xmin of detected face wrt to original image
        ymin = int(bboxes[filename].ymin*image.shape[0]) # find the ymin of the detected face wrt to original image
        pred_x_new, pred_y_new = xmin + pred_x, ymin + pred_y #the coordiantes of landmarks on the original image

        output_file = os.path.join(output_path, filename) 

        for idx,point in enumerate(zip(pred_x_new, pred_y_new)):
            image = cv2.circle(image, point, radius=6, color=(0, 0, 255), thickness=-1) #save the original image with landmark on it
        
        cv2.imwrite(output_file, image)

In [6]:
def run_dad_net_more_points(image_path, num_points):

    # keypoint dir contain the index of each keypoints that we want to consider for our landmarks. 
    # Only these points are extracted from the projected list of indices

    if (num_points == 445):
        keypoint_dir = "model_training/model/static/face_keypoints/keypoints_445/"
    elif(num_points == 191):
        keypoint_dir = "model_training/model/static/face_keypoints/keypoints_191/"
    else:
        ValueError("Invalid keypoints subset provided.\n"
                   "Available options are: 191, 445")
    
    image = read_rgb_image(image_path)
    predictor = FaceMeshPredictor.dad_3dnet()
    predictions = predictor(image)

    subset_dir = []
    for file in os.listdir(keypoint_dir):
        subset_dir.append(os.path.join(keypoint_dir, file))
    projected_vertices = predictions["projected_vertices"].squeeze().numpy().astype(int) # projected vertices (total of 5023)
    #print(projected_vertices.shape)
    points = []
#print(subset_dir)
    for subs_file in subset_dir:
        #subs_file_path = os.path.join(subset_dir, subs_file)
        #print(len(np.take(projected_vertices, load_indices_from_npy(subs_file), axis=0)))
        points.extend(np.take(projected_vertices, load_indices_from_npy(subs_file), axis=0))
    
    pred_x,pred_y = [], []
    for point in points:
        pred_x.append(point[0])
        pred_y.append(point[1])
    
    ### this is to test all the 5023 points. comment out below two lines

    #pred_x = projected_vertices[:,0]
    #pred_y = projected_vertices[:,1]
    
    return np.array(pred_x), np.array(pred_y)

In [7]:
def run_dadnet_on_large_images_more_points(original_images, data_path, output_path, bboxes, n_points):
    #data_path is the path to the detected faces images (not the full image)
    IMAGE_FILES = []

    files = os.listdir(data_path)

    for file in files:
        curr_image = os.path.join(data_path, file)
        IMAGE_FILES.append(curr_image)

    for idx, file in enumerate(IMAGE_FILES):
        pred_x,pred_y = run_dad_net_more_points(file,n_points)

        filename = os.path.basename(file)
        original_image = os.path.join(original_images, filename)
        image = cv2.imread(original_image)
        xmin = int(bboxes[filename].xmin*image.shape[1])
        ymin = int(bboxes[filename].ymin*image.shape[0])
        pred_x_new, pred_y_new = xmin + pred_x, ymin + pred_y

        output_file = os.path.join(output_path, filename)

        for idx,point in enumerate(zip(pred_x_new, pred_y_new)):
            image = cv2.circle(image, point, radius=3, color=(0, 0, 255), thickness=-1)
        
        cv2.imwrite(output_file, image)

#### first find the bounding boxes

In [27]:
facescape_sample5 = "../Dataset/facescape_sample/sample5/512/1/"
detected_path = "../Dataset/facescape_sample/sample5/512_detected/1"
bounding_boxes_facescape = mediapipe_face_detection(facescape_sample5,detected_path)

55


##### run for 68 landmarks

In [12]:
output_path = "./outputs/facescape_sample/sample3/detect_mp_dad_68/"
run_dadnet_on_large_images(facescape_sample2,detected_path, output_path, bounding_boxes_facescape)

/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return forward_call(*input, **kwargs)
/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return forward_call(*input, **kwargs)


<class 'dict'>
<class 'dict'>


##### run for 191/445 landmarks

In [22]:
output_path = "./outputs/facescape_sample/sample3/detect_mp_dad_191/"
run_dadnet_on_large_images_more_points(facescape_sample2,detected_path, output_path, bounding_boxes_facescape,191)

##### landmarks for custom index list

In [8]:
def run_dad_net_custom_points(image_path, indices_file):

    # pass a numpy indices to get those points 
    
    image = read_rgb_image(image_path)
    predictor = FaceMeshPredictor.dad_3dnet()
    predictions = predictor(image)
    projected_vertices = predictions["projected_vertices"].squeeze().numpy().astype(int) # projected vertices (total of 5023)
    #print(projected_vertices.shape)
    indices = np.load(indices_file)
    points = []
    points.extend(np.take(projected_vertices, indices, axis=0))
    
    pred_x,pred_y = [], []
    for point in points:
        pred_x.append(point[0])
        pred_y.append(point[1])
    
    return np.array(pred_x), np.array(pred_y)

In [9]:
def run_dadnet_on_large_images_custom_points(original_images, data_path, output_path, bboxes, indices_file):
    #data_path is the path to the detected faces images (not the full image)
    IMAGE_FILES = []

    files = os.listdir(data_path)

    for file in files:
        curr_image = os.path.join(data_path, file)
        IMAGE_FILES.append(curr_image)

    for idx, file in enumerate(IMAGE_FILES):
        pred_x,pred_y = run_dad_net_custom_points(file,indices_file)

        filename = os.path.basename(file)
        original_image = os.path.join(original_images, filename)
        image = cv2.imread(original_image)
        xmin = int(bboxes[filename].xmin*image.shape[1])
        ymin = int(bboxes[filename].ymin*image.shape[0])
        pred_x_new, pred_y_new = xmin + pred_x, ymin + pred_y

        output_file = os.path.join(output_path, filename)
        lmk_file = os.path.join(output_path,os.path.basename(filename).split(".")[0] + ".npy")
        lmks = []
        for idx,point in enumerate(zip(pred_x_new, pred_y_new)):
            lmks.append(point)
            image = cv2.circle(image, point, radius=3, color=(0, 0, 255), thickness=-1)
        
        lmks = np.array(lmks)
        #print(lmk_file)
        np.save(lmk_file, lmks)
        #cv2.imwrite(output_file, image)

In [31]:
output_path = "./outputs/facescape_sample/sample5/1/"
indices_file = "./model_training/model/static/face_keypoints/ids1.npy"
run_dadnet_on_large_images_custom_points(facescape_sample5,detected_path, output_path, bounding_boxes_facescape,indices_file)

/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return forward_call(*input, **kwargs)
/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return forward_call(*input, **kwargs)


In [33]:
landmarks = np.load("./outputs/facescape_sample/sample3/detect_mp_dad_1797/12.npy")

In [36]:
landmarks.shape

(1797, 2)

### Run mp_dad on multiple folders

In [35]:
full_dataset = "../Dataset/facescape_sample/sample5/512/"
detected_path = "../Dataset/facescape_sample/sample5/512_detected/"
bounding_boxes_facescape = {}
for path in os.listdir(full_dataset):
    #print(path)
    curr_facescape_sample = os.path.join(full_dataset, path)
    curr_detected_path = os.path.join(detected_path, path)
    if not os.path.exists(curr_detected_path):
        os.makedirs(curr_detected_path)
    bounding_boxes_facescape[path] = mediapipe_face_detection(curr_facescape_sample,curr_detected_path)

20
66
18
59
341
57
9
59
11
68
7
57
16
57
6
56
17
58
1
55
340
55
19
57
121
56
212
57
342
57
4
54
15
68
3
56
122
59
2
57
13
57
5
58
14
68
344
56
343
57


In [40]:
output_path = "./outputs/facescape_sample/sample5/"
indices_file = "./model_training/model/static/face_keypoints/ids1.npy"
for path in os.listdir(detected_path):
    curr_facescape_sample = os.path.join(full_dataset, path)
    curr_detected_path = os.path.join(detected_path, path)
    curr_output_path = os.path.join(output_path, path)
    if not os.path.exists(curr_output_path):
        os.makedirs(curr_output_path)
    run_dadnet_on_large_images_custom_points(curr_facescape_sample,curr_detected_path, curr_output_path, bounding_boxes_facescape[path],indices_file)

In [42]:
landmarks = np.load("./outputs/facescape_sample/sample5/1/1.npy")
print(landmarks.shape)

(1797, 2)


In [43]:
image = cv2.imread("../Dataset/facescape_sample/sample5/512/1/1.jpg")
for point in landmarks:
    image = cv2.circle(image, point, radius=3, color=(0, 0, 255), thickness=-1)

cv2.imwrite('test1.jpg', image)

True

In [10]:
!pwd

/Users/rajakumar/Desktop/MS/Fall-2022/research/2D_Landmarks/DAD-3DHeads


#### run for stirling and H3DS test dataset

In [12]:
full_dataset ='../Dataset/H3DS_sample/test_set/H3DS/'
detected_path = "../Dataset/H3DS_sample/test_set/H3DS_detected/"
bounding_boxes_h3ds = {}
for path in os.listdir(full_dataset):
    #print(path)
    curr_hds_sample = os.path.join(full_dataset, path)
    curr_detected_path = os.path.join(detected_path, path)
    if not os.path.exists(curr_detected_path):
        os.makedirs(curr_detected_path)
    bounding_boxes_h3ds[path] = mediapipe_face_detection(curr_hds_sample,curr_detected_path)

71


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


69
68
68
69
70
68
69
70
68
69
65
68
69
68
66
69
68
73
70


In [14]:
output_path = "./outputs/H3DS/test_set/"
indices_file = "./model_training/model/static/face_keypoints/ids1.npy"
for path in os.listdir(detected_path):
    curr_hds_sample = os.path.join(full_dataset, path)
    curr_detected_path = os.path.join(detected_path, path)
    curr_output_path = os.path.join(output_path, path)
    if not os.path.exists(curr_output_path):
        os.makedirs(curr_output_path)
    run_dadnet_on_large_images_custom_points(curr_hds_sample,curr_detected_path, curr_output_path, bounding_boxes_h3ds[path],indices_file)

/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return forward_call(*input, **kwargs)
/Users/rajakumar/miniforge3/envs/dad_3d/lib/python3.8/site-packages/torch/nn/modules/module.py:1051: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return forward_call(*input, **kwargs)


### visualize the output

In [7]:
lmk_path = "./outputs/H3DS/test_set/0cd3f3c0bc34a287/"
original_data_path = "../Dataset/H3DS_sample/test_set/H3DS/0cd3f3c0bc34a287/"

for data in os.listdir(lmk_path):

    filename = data.split(".")[0]
    curr_lmk = os.path.join(lmk_path, filename + ".npy")
    curr_img = os.path.join(original_data_path, filename+'.jpg')
    output_path = os.path.join(lmk_path, filename + '.jpg')
    #print(output_path)
    landmarks = np.load(curr_lmk)
    #print(landmarks.shape)
    image = cv2.imread(curr_img)
    for point in landmarks:
        image = cv2.circle(image, point, radius=3, color=(0, 0, 255), thickness=-1)

    cv2.imwrite(output_path, image)

In [16]:
full_dataset ='../Dataset/stirling/stirling_test/'
detected_path = "../Dataset/stirling/test_detected/"
bounding_boxes_h3ds = {}
for path in os.listdir(full_dataset):
    #print(path)
    curr_hds_sample = os.path.join(full_dataset, path)
    curr_detected_path = os.path.join(detected_path, path)
    if not os.path.exists(curr_detected_path):
        os.makedirs(curr_detected_path)
    bounding_boxes_h3ds[path] = mediapipe_face_detection(curr_hds_sample,curr_detected_path)

In [17]:
output_path = "./outputs/stirling/test_set/"
indices_file = "./model_training/model/static/face_keypoints/ids1.npy"
for path in os.listdir(detected_path):
    curr_hds_sample = os.path.join(full_dataset, path)
    curr_detected_path = os.path.join(detected_path, path)
    curr_output_path = os.path.join(output_path, path)
    if not os.path.exists(curr_output_path):
        os.makedirs(curr_output_path)
    run_dadnet_on_large_images_custom_points(curr_hds_sample,curr_detected_path, curr_output_path, bounding_boxes_h3ds[path],indices_file)

In [8]:
lmk_path = "./outputs/stirling/test_set/F1003/"
original_data_path = "../Dataset/stirling/stirling_test/F1003/"

for data in os.listdir(lmk_path):

    filename = data.split(".")[0]
    curr_lmk = os.path.join(lmk_path, filename + ".npy")
    curr_img = os.path.join(original_data_path, filename+'.jpg')
    output_path = os.path.join(lmk_path, filename + '.jpg')
    #print(output_path)
    landmarks = np.load(curr_lmk)
    #print(landmarks.shape)
    image = cv2.imread(curr_img)
    for point in landmarks:
        image = cv2.circle(image, point, radius=3, color=(0, 0, 255), thickness=-1)

    cv2.imwrite(output_path, image)